In [1]:
using TSSM

 *** OPENMP n_threads =           4


In [2]:
nx = 512
xmin = -16
xmax = +16
meth = Schroedinger1D(nx, xmin, xmax, cubic_coupling=-1)

TSSM.Schroedinger1D{Float64}(Ptr{Void} @0x0000000003ecf220)

In [3]:
# cubic_coupling=-1 has to be multiplied by -1im 
# because of the factor 1im at the lefthand side of the Schrödinger equation
F(t, x, u) = 1im*conj(u)*u^2
function gen_rhs!(rhs::WfSchroedinger1D, psi::WfSchroedinger1D)
    to_real_space!(rhs)
    to_real_space!(psi)
    t = get_time(psi)
    u = get_data(psi, true)
    u1 = get_data(rhs, true)
    x = get_nodes(psi.m)
    for j=1:length(u)
        u1[j] = F(t, x[j], u[j])
    end
end

gen_rhs! (generic function with 1 method)

In [4]:
# exact solution
const a = 2
const b = 1
const c = 0
function soliton(x, t)
    h = (a^2 - b^2)/2*t - b*x
    (a./cosh(a*(b*t+x-c))).*exp(1im*h) 
end    

soliton (generic function with 1 method)

In [5]:
t0 = 0
tend = 1
psi = wave_function(meth)
psi_ref = wave_function(meth)
set!(psi, soliton, t0)        # initial data at t=t0
set!(psi_ref, soliton, tend)  # reference solution at t=tend

In [2]:
include("exponential_multistep.jl")

global_orders (generic function with 1 method)

In [7]:
set!(psi, soliton, t0)
dt=0.01
for t in exponential_multistep_equidistant_time_stepper(psi, t0, tend, dt, 
    4, N2=0, steps=10, iter=0, combine_first=true)
end    
get_time(psi)
set!(psi_ref, soliton, get_time(psi))
distance(psi, psi_ref)

3.0869508403203638e-6

In [23]:
set!(psi, soliton, t0)
set!(psi_ref, soliton, tend)
global_orders(psi, psi_ref, t0, tend, .25, 7, N2=8, rows=10, iter=1, combine_first=false)

             dt         err      p
-----------------------------------
  1   2.500e-01   3.515e+00
  2   1.250e-01   2.231e-02   7.30
  3   6.250e-02   1.671e-04   7.06
  4   3.125e-02   8.818e-07   7.57
  5   1.563e-02   4.186e-09   7.72
  6   7.813e-03   1.810e-11   7.85
  7   3.906e-03   2.262e-13   6.32
  8   1.953e-03   2.286e-13  -0.02
  9   9.766e-04   2.355e-13  -0.04
 10   4.883e-04   3.179e-13  -0.43


In [9]:
set!(psi, soliton, t0)
set!(psi_ref, soliton, tend)
global_orders(psi, psi_ref, t0, tend, 1/8, 4, rows=12, iter=0, combine_first=true)

             dt         err      p
-----------------------------------
  1   1.250e-01   2.044e-01
  2   6.250e-02   1.660e-02   3.62
  3   3.125e-02   1.197e-03   3.79
  4   1.563e-02   7.883e-05   3.92
  5   7.813e-03   5.032e-06   3.97
  6   3.906e-03   3.174e-07   3.99
  7   1.953e-03   1.992e-08   3.99
  8   9.766e-04   1.247e-09   4.00
  9   4.883e-04   7.778e-11   4.00
 10   2.441e-04   4.240e-12   4.20
 11   1.221e-04   1.517e-12   1.48
 12   6.104e-05   2.593e-12  -0.77


In [10]:
α=0.5
#V2(x,t) = (1-α)*κ*abs(soliton(x,t)).^2
method = Schroedinger1D(nx, xmin, xmax, cubic_coupling=-α)
psi = wave_function(method)
psi_ref = wave_function(method)
set!(psi, soliton, t0)        # initial data at t=t0
set!(psi_ref, soliton, tend)  # reference solution at t=tend

In [11]:
# cubic_coupling=-1 has to be multiplied by -1im 
# because of the factor 1im at the lefthand side of the Schrödinger equation
F(t, x, u) = α*1im*conj(u)*u^2 + (1-α)*1im*a^2./cosh(a*(b*t+x-c)).^2*u
function gen_rhs!(rhs::WfSchroedinger1D, psi::WfSchroedinger1D)
    to_real_space!(rhs)
    to_real_space!(psi)
    t = get_time(psi)
    u = get_data(psi, true)
    u1 = get_data(rhs, true)
    x = get_nodes(psi.m)
    for j=1:length(u)
        u1[j] = F(t, x[j], u[j])
    end
end

gen_rhs! (generic function with 1 method)

In [12]:
set!(psi, soliton, t0)
dt=0.01
for t in exponential_multistep_equidistant_time_stepper(psi, t0, tend, dt, 4, N2=5, steps=10, iter=1)
end    
get_time(psi)
set!(psi_ref, soliton, get_time(psi))
distance(psi, psi_ref)

1.8296807323231395e-7

In [13]:
set!(psi, soliton, t0)
set!(psi_ref, soliton, tend)
global_orders(psi, psi_ref, t0, tend, 1/8, 4, rows=8, iter=0)

             dt         err      p
-----------------------------------
  1   1.250e-01   9.196e-02
  2   6.250e-02   6.790e-03   3.76
  3   3.125e-02   4.759e-04   3.83
  4   1.563e-02   3.114e-05   3.93
  5   7.813e-03   1.983e-06   3.97
  6   3.906e-03   1.248e-07   3.99
  7   1.953e-03   7.824e-09   4.00
  8   9.766e-04   4.896e-10   4.00


In [14]:
get_time(psi)

0.0

In [15]:
V2 = Matrix{Float64}(inv(Rational{Int}[n^m//factorial(m) for n=-4+3:2, m=0:4-1]))

4x4 Array{Float64,2}:
  0.0        1.0   0.0   0.0     
 -0.333333  -0.5   1.0  -0.166667
  1.0       -2.0   1.0   0.0     
 -1.0        3.0  -3.0   1.0     

In [17]:
function phi(x, n)
    if n==0
        return exp(x)
    end
    if abs(x)<=0.5
        if n==1
            m=15
        elseif n==2
            m=14
        elseif n<=5
            m=13
        elseif n<=8
            m=12
        else
            m=11
        end
        h = 1.0
        for k=1:m
            h = h*x/(1+n+m-k) +1
        end
        fac_k = 1
        for k=1:n
            fac_k = k*fac_k
        end 
        return h/factorial(n)        
    else
        h = expm1(x)/x
        fac_k = 1            
        for k=1:n-1
            fac_k = k*fac_k
            h = (h - 1.0/fac_k)/x
        end
        return h
    end
end 

phi (generic function with 1 method)

In [7]:
V(x) = x^2
meth = SchroedingerReal1D(1024, -30, 30, potential=V, cubic_coupling=390)
psi=wave_function(meth)

TSSM.WfSchroedingerReal1D{Float64}(Ptr{Void} @0x0000000002c23ca0,TSSM.SchroedingerReal1D{Float64}(Ptr{Void} @0x0000000002e24c00))

In [8]:
f(x)=exp(-x^2)
set!(psi, f)

In [9]:
include("groundstate.jl")

groundstate! (generic function with 1 method)

In [11]:
groundstate!(psi, extrapolation_order=2)
save(psi, "exponential_multistep_test_groundstate.h5")

    0   4.112629113735760e+01   7.411670189794734e+01   1.000e+06   5.522e-01   1.040e+01      0.01   2.850224319614514e+01   1.000e+06   3.292e-01
changed step size, old:   5.000000000000000e-02  new:   2.500000000000000e-02
    1   2.688007443969697e+01   4.313509833989843e+01   1.425e+01   1.212e-01   1.441e-01      0.01
    2   2.707612519029218e+01   4.457455820947956e+01  -1.961e-01   2.659e-01   8.340e-02      0.01
    3   2.848453155357161e+01   4.747852104389232e+01  -1.408e+00   5.565e-01   8.648e-02      0.02
    4   2.869136968454695e+01   4.797070685986320e+01  -2.068e-01   6.238e-01   4.741e-02      0.02
    5   2.859879056224278e+01   4.763005242021936e+01   9.258e-02   6.185e-01   3.399e-02      0.02
    6   2.817209466007018e+01   4.675742220720481e+01   4.267e-01   5.680e-01   3.161e-02      0.02
    7   2.801001690433026e+01   4.651995934489804e+01   1.621e-01   5.374e-01   1.309e-02      0.02
    8   2.801623427178704e+01   4.656645272833871e+01  -6.217e-03   5.403e

In [3]:
V(x) = x^2
meth = Schroedinger1D(1024, -30, 30, potential=V, cubic_coupling=390)
psi=wave_function(meth)

TSSM.WfSchroedinger1D{Float64}(Ptr{Void} @0x00000000033f1ca0,TSSM.Schroedinger1D{Float64}(Ptr{Void} @0x0000000003394c70))

In [4]:
F(t, x, u) = -390im*conj(u)*u^2 -1im*V(x)*u
function gen_rhs!(rhs::WfSchroedinger1D, psi::WfSchroedinger1D)
    to_real_space!(rhs)
    to_real_space!(psi)
    t = get_time(psi)
    u = get_data(psi, true)
    u1 = get_data(rhs, true)
    x = get_nodes(psi.m)
    for j=1:length(u)
        u1[j] = F(t, x[j], u[j])
    end
end

gen_rhs! (generic function with 1 method)

In [18]:
t0=0
tend=10
steps = 12000
dt = (tend-t0)/steps
time0 = time()
load!(psi, "exponential_multistep_test_groundstate.h5");
set_time!(psi, 0)
for t in  exponential_multistep_equidistant_time_stepper(psi, t0, tend, dt, 
    4, N2=4, steps=steps, iter=1, combine_first=true, final_iteration=true)
    nn = norm(psi)
    if abs(nn-1.0)>2
            break
    end
    if mod(t,100)==0
        nn = norm(psi)
        E_pot = potential_energy(psi)
        E_kin = kinetic_energy(psi)
        E_int = interaction_energy(psi)
        E_tot = E_pot+E_kin+E_int
        @printf("step=%5i  t=%14.10f  norm=%14.10f  E_pot=%14.10f  E_kin=%14.10f  E=%14.10f  ctime=%10.2f\n", 
            t, get_time(psi), nn, E_pot, E_kin, E_tot, time()-time0)
    end
end

step=  100  t=  0.0833333333  norm=  0.9999993578  E_pot=  8.8315132976  E_kin=  0.0267026247  E= 26.4678147460  ctime=      0.39
step=  200  t=  0.1666666667  norm=  0.9999993973  E_pot=  8.8315135201  E_kin=  0.0267026283  E= 26.4678182315  ctime=      0.55
step=  300  t=  0.2500000000  norm=  0.9999994369  E_pot=  8.8315134695  E_kin=  0.0267026330  E= 26.4678217170  ctime=      0.71
step=  400  t=  0.3333333333  norm=  0.9999994764  E_pot=  8.8315131961  E_kin=  0.0267026388  E= 26.4678252025  ctime=      0.87
step=  500  t=  0.4166666667  norm=  0.9999995159  E_pot=  8.8315127598  E_kin=  0.0267026452  E= 26.4678286880  ctime=      1.03
step=  600  t=  0.5000000000  norm=  0.9999995555  E_pot=  8.8315122269  E_kin=  0.0267026519  E= 26.4678321736  ctime=      1.19
step=  700  t=  0.5833333333  norm=  0.9999995950  E_pot=  8.8315116678  E_kin=  0.0267026581  E= 26.4678356591  ctime=      1.35
step=  800  t=  0.6666666667  norm=  0.9999996346  E_pot=  8.8315111542  E_kin=  0.0267026